In [176]:
import numpy as np
import pandas as pd

In [177]:
def sharpe_ratio(daily_returns):
    # get volatility
    vola_daily = np.std(daily_returns)
    # get returns
    total_returns = np.product(1 + daily_returns)
    returns_daily = np.exp(np.log(total_returns)/daily_returns.shape[0]) - 1
    return returns_daily / vola_daily
    
def sortino_ratio(daily_returns):
    # get volatility
    downsideReturns = daily_returns.copy()
    downsideReturns[downsideReturns > 0] = 0
    downsideVola = np.std(downsideReturns)
    # get returns
    total_returns = np.product(1 + daily_returns)
    returns_daily = np.exp(np.log(total_returns)/daily_returns.shape[0]) - 1
    return returns_daily / downsideVola

def slippage_costs(high, low, close, delta_weights, factor=0.05):
    """Taken as (High - Low) * factor, measured in absolute costs."""
    slippage = (high - low) / close * 0.05
    return abs(delta_weights * slippage)

def single_sim(data, old_weights, new_weights, portfolio_val, pred_prices, index):
    """Computes the estimated sharpe ratio for tomorrow given weights and predictions."""
    # index of today for convenience of accessing data's values
    TODAY, YTD = index, index-1
    
    # today's sharpe ratio (compute using past 5 days returns/std dev)
    # array of each stock's pct return today
    today_ret = (data.loc[TODAY, 'CLOSE'] - data.loc[YTD, 'CLOSE']) / data.loc[YTD, 'CLOSE']
    # array of weighted stock pct returns
    today_portfolio_ret = old_weights * today_ret
    # add today's new portfolio value to Series of portfolio values
    portfolio_val = portfolio_val.append(
        pd.Series(np.sum(today_portfolio_ret) * portfolio_val.iloc[-1])
    )
    # sharpe of pct returns over past 5 days (inc. today)
    today_sharpe = sharpe_ratio(portfolio_val.pct_change().tail(5))
    
    delta_weights = new_weights - old_weights
    # Use today's high-low to estimate slippage tomorrow
    slippage = slippage_costs(data.loc[TODAY, 'HIGH'], 
                              data.loc[TODAY, 'LOW'],
                              data.loc[YTD, 'CLOSE'],
                              delta_weights)
    
    # find tomorrow's returns based on new weights and predicted prices
    # ASSUMPTION: tomorrow's OPEN = today's CLOSE
    # array of each stock's predicted pct return tomorrow
    pred_ret = (pred_prices - data.loc[TODAY, 'CLOSE']) / data.loc[TODAY, 'CLOSE'] # % change
    # array of predicted weighted stock pct returns, deducting slippage for each stock
    pred_portfolio_ret = new_weights * pred_ret - slippage
    # sharpe of past 4 days (inc. today) and tomorrows predicted portfolio return
    pred_sharpe = sharpe_ratio(
        portfolio_val.append(
            pd.Series(np.sum(pred_portfolio_ret) * portfolio_val.iloc[-1]))\
                     .pct_change()\
                     .tail(5)
    )
    # maximize the difference here to greedily optimize
    return pred_sharpe - today_sharpe, pred_sharpe, pred_portfolio_ret, pred_sharpe


Below cells are for testing purposes to make sure it runs correctly.

In [151]:
df = pd.read_pickle('../data/data_base_test.pkl').fillna(method='ffill').fillna(method='bfill')[['F_AD', 'F_AE', 'F_C']]

In [152]:
df.columns = df.columns.swaplevel()

In [153]:
old = np.random.normal(size=3)
new = np.random.normal(size=3)
portfolio_val = pd.Series([1e6] * df.shape[0])
pred_prices = df.tail(1)['CLOSE']
increased_sharpe, pred_sharpe, pred_portfolio_ret, pred_sharpe = single_sim(df.reset_index(drop=True), 
                                                                            old, new, portfolio_val, 
                                                                            pred_prices, 30)

In [154]:
increased_sharpe

1.332609804509306

In [155]:
pred_sharpe

-0.00596514345982034

In [156]:
pred_portfolio_ret

TICKER,F_AD,F_AE,F_C
DATE,,,
2017-12-29,-0.002698,-0.900013,0.01441


In [157]:
np.asarray(pred_portfolio_ret).reshape(-1)

array([-0.00269814, -0.90001274,  0.01441012])

In [158]:
pred_sharpe 

-0.00596514345982034

* Budget = SGD1000
* Stock A: SGD100 0.5 = 5 units,   predict SGD110 new_weight=0.6 = 6 units, est. profits=SGD60
* Stock B: SGD200 0.5 = 2.5 units, predict SGD250 new_weight=0.4 = 2 units, est. profits=SGD100
* True profit = (new_weight_A x Budget x price_change_A) + (new_weight_B x Budget x price_change_B)

Questions:
* How to set initial portfolio weights?

# Steps for greedy MC portfolio optimization

__Given predictions for next day + current portfolio weights, how to reweight the portfolio for Max. Sharpe__

(Naive)
1. Create random set of (delta) weights
2. Calculate total slippage cost (check formula in runts)
3. Using random weights, compute new portfolio estimated returns
4. Use past 5 days returns, find current sharpe ratio
5. Add next day's returns, find new sharpe ratio
6. Maximise difference in the ratios

# TODO: Create function to Monte Carlo simulate every day
(use single_sim for each day, then pick best set of weights)

In [167]:
def mc_simulation(portfolio, old_weights, portfolio_val, pred_prices, 
                  index, random_state=12345, num_sims=1000000):
    # i suppose here hte portfolio is the same as data used in single_sim
    # portfolio_val: list of hist portfolio values 
    # names: names of futures
    
    #set random seed for reproduction's sake
    np.random.seed(12345)
    # set the number of combinations for imaginary portfolios
    num_assets = old_weights.shape[0]
    print("number of assets:", num_assets)
    df = portfolio
    
    df.columns = df.columns.swaplevel()
    print(df.head())
    
    #port_returns = []
    #port_volatility = []
    increased_sharpe_ratios = []
    sharpe_ratios = []
    stock_weights = []

    # populate the empty lists with each portfolios returns,risk and weights
    for single_portfolio in range(num_sims):
        # new weights
        weights = np.random.normal(size=num_assets)
        new_weights = weights / np.sum(np.absolute(weights))
        
        increased_sharpe, pred_sharpe, pred_portfolio_ret, pred_sharpe = single_sim(df.reset_index(drop=True), 
                                                             old_weights, new_weights, portfolio_val, 
                                                             pred_prices, index)
        stock_weights.append(new_weights)
        increased_sharpe_ratios.append(increased_sharpe)
        sharpe_ratios.append(pred_sharpe)
    
    return stock_weights, increased_sharpe_ratios, sharpe_ratios


In [168]:
stock_weights, increased_sharpe_ratios, sharpe_ratios = mc_simulation(portfolio=df, old_weights=old, portfolio_val=portfolio_val, 
              pred_prices=pred_prices, index=30, random_state=12345, num_sims=100)

number of assets: 3
PRICE          OPEN     HIGH      LOW    CLOSE     OPEN     HIGH      LOW  \
TICKER         F_AD     F_AD     F_AD     F_AD     F_AE     F_AE     F_AE   
DATE                                                                        
2016-01-01  72690.0  72750.0  71310.0  71580.0  87590.0  87600.0  85400.0   
2016-01-04  72690.0  72750.0  71310.0  71580.0  87590.0  87600.0  85400.0   
2016-01-05  71570.0  71900.0  71090.0  71270.0  87130.0  87420.0  85700.0   
2016-01-06  71360.0  71480.0  70250.0  70370.0  86390.0  86750.0  84840.0   
2016-01-07  70400.0  70640.0  69580.0  69730.0  84260.0  84640.0  82340.0   

PRICE         CLOSE     OPEN     HIGH      LOW    CLOSE  
TICKER         F_AE      F_C      F_C      F_C      F_C  
DATE                                                     
2016-01-01  86320.0  17975.0  18000.0  17525.0  17575.0  
2016-01-04  86320.0  17975.0  18000.0  17525.0  17575.0  
2016-01-05  86970.0  17638.0  17800.0  17588.0  17650.0  
2016-01-06  857

/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: RuntimeWarning: invalid value encountered in log


In [170]:
stock_weights, increased_sharpe_ratios, sharpe_ratios

([array([-0.17015162,  0.39809445, -0.43175393]),
  array([-0.14195201,  0.50212577,  0.35592222]),
  array([0.08123614, 0.24635124, 0.67241261]),
  array([ 0.35112369,  0.28372768, -0.36514863]),
  array([0.14809806, 0.12328212, 0.72861982]),
  array([ 0.27191842, -0.61401628, -0.11406531]),
  array([ 0.63045454, -0.16566452, -0.20388094]),
  array([ 0.10047805,  0.6843979 , -0.21512405]),
  array([-0.57488999,  0.12364867,  0.30146133]),
  array([2.80313547e-01, 5.03220034e-04, 7.19183233e-01]),
  array([-0.1822623 , -0.2123036 , -0.60543411]),
  array([-0.56701755, -0.26229301,  0.17068944]),
  array([-0.5168611 ,  0.04892353, -0.43421537]),
  array([ 0.11511086, -0.81588717, -0.06900197]),
  array([-0.40368893, -0.2541352 , -0.34217586]),
  array([ 0.20190779,  0.26652001, -0.5315722 ]),
  array([0.18920829, 0.77088281, 0.03990891]),
  array([ 0.19525439, -0.00938903,  0.79535658]),
  array([ 0.34967006, -0.24219375, -0.40813618]),
  array([ 0.01539451,  0.52624928, -0.45835621]),


Sample code for MC portfolio optimization structure below:

In [9]:
# empty lists to store returns, variance and weights of imiginary portfolios
port_returns = []
port_volatility = []
sharpe_ratio = []
stock_weights = []

# set the number of combinations for imaginary portfolios
num_assets = portfolio.shape[1]
num_portfolios = 100000

#set random seed for reproduction's sake
np.random.seed(12345)

# populate the empty lists with each portfolios returns,risk and weights
for single_portfolio in range(num_portfolios):
    weights = np.random.random(num_assets)
    print(weights)
    weights /= np.sum(weights)
    returns = np.dot(weights, daily_mean)
    volatility = np.sqrt(np.dot(weights, np.dot(daily_cov, weights)))
    sharpe = (returns - rf)/ volatility
    
    sharpe_ratio.append(sharpe)
    port_returns.append(returns)
    port_volatility.append(volatility)
    stock_weights.append(weights)

# a dictionary for Returns and Risk values of each portfolio
results = {'Returns': port_returns,
           'Volatility': port_volatility,
           'Sharpe Ratio': sharpe_ratio}

# extend original dictionary to accomodate each ticker and weight in the portfolio
for i, symbol in enumerate(names):
    results[symbol+' Weight'] = [Weight[i] for Weight in stock_weights]

NameError: name 'portfolio' is not defined

### Another possible way - run optimisation directly 

In [182]:
df = pd.read_pickle('../data/data_base_test.pkl').fillna(method='ffill').fillna(method='bfill')[['F_AD', 'F_AE', 'F_C']]

In [183]:
df.columns = df.columns.swaplevel()
old = np.random.normal(size=3)
new = np.random.normal(size=3)
portfolio_val = pd.Series([1e6] * df.shape[0])
pred_prices = df.tail(1)['CLOSE']

In [184]:
def single_sim_sharpe(portfolio, new_weights, old_weights, portfolio_val, pred_prices, index):
    """copy from the above, except return the sharpe instead of the difference"""
    """Computes the estimated sharpe ratio for tomorrow given weights and predictions."""
    data = portfolio
     
    # index of today for convenience of accessing data's values 
    TODAY, YTD = index, index-1
    
    # today's sharpe ratio (compute using past 5 days returns/std dev)
    # array of each stock's pct return today
    today_ret = (data.loc[TODAY, 'CLOSE'] - data.loc[YTD, 'CLOSE']) / data.loc[YTD, 'CLOSE']
    # array of weighted stock pct returns
    today_portfolio_ret = old_weights * today_ret
    # add today's new portfolio value to Series of portfolio values
    portfolio_val = portfolio_val.append(
        pd.Series(np.sum(today_portfolio_ret) * portfolio_val.iloc[-1])
    )
    # sharpe of pct returns over past 5 days (inc. today)
    today_sharpe = sharpe_ratio(portfolio_val.pct_change().tail(5))
    
    delta_weights = new_weights - old_weights
    # Use today's high-low to estimate slippage tomorrow
    slippage = slippage_costs(data.loc[TODAY, 'HIGH'], 
                              data.loc[TODAY, 'LOW'],
                              data.loc[YTD, 'CLOSE'],
                              delta_weights)
    
    # find tomorrow's returns based on new weights and predicted prices
    # ASSUMPTION: tomorrow's OPEN = today's CLOSE
    # array of each stock's predicted pct return tomorrow
    pred_ret = (pred_prices - data.loc[TODAY, 'CLOSE']) / data.loc[TODAY, 'CLOSE'] # % change
    # array of predicted weighted stock pct returns, deducting slippage for each stock
    pred_portfolio_ret = new_weights * pred_ret - slippage
    # sharpe of past 4 days (inc. today) and tomorrows predicted portfolio return
    pred_sharpe = sharpe_ratio(
        portfolio_val.append(
            pd.Series(np.sum(pred_portfolio_ret) * portfolio_val.iloc[-1]))\
                     .pct_change()\
                     .tail(5)
    )
    # maximize the difference here to greedily optimize
    return pred_sharpe  

In [188]:
from scipy.optimize import minimize

def neg_obj_function(new_weights, data, old_weights, portfolio_val, pred_prices, index):
    return -single_sim_sharpe(data, old_weights, new_weights, 
                       portfolio_val, pred_prices, index)

def max_diff_sharpe_ratio(data, old_weights, portfolio_val, pred_prices, index):
    num_assets = old_weights.shape[0]
    args = (data, old_weights, portfolio_val, pred_prices, index)
    
    # constraint: sum of weight = 1
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    # for each future, search from -1 to 1
    bound = (-1.0, 1.0)
    bounds = tuple(bound for asset in range(num_assets))
    
    # run optimisation:
    # maximize the sharpe = minimize the neg value of it
    result = minimize(neg_obj_function, num_assets*[1./num_assets,], 
                      args=args, method='SLSQP', 
                      bounds=bounds, constraints=constraints)
    return result 

In [189]:
max_diff_sharpe_ratio(df.reset_index(drop=True), old, portfolio_val, pred_prices, 30 )

     fun: 0.2800502281986486
     jac: array([ 0.0005504 , -0.01603321,  0.00311245])
 message: 'Optimization terminated successfully.'
    nfev: 110
     nit: 19
    njev: 19
  status: 0
 success: True
       x: array([ 3.8139954e-07,  1.0000000e+00, -3.8139954e-07])

In [190]:
np.absolute(np.asarray([1,-1,1]))

array([1, 1, 1])